In [1]:
import pandas as pd
import numpy as np

In [2]:
file=pd.DataFrame()

In [3]:
file

""


In [4]:
for i in range(10):
    f1=pd.read_csv('/home/yazi/Documents/stage/data/car_afc_train/train'+str(i)+'_first.csv',delimiter=';',encoding='iso 8859-1',dtype=str)
    f2=pd.read_csv('/home/yazi/Documents/stage/data/car_afc_train/train'+str(i)+'_last.csv',delimiter=';',encoding='iso 8859-1',dtype=str)
    file=pd.concat([f1,f2,file],ignore_index=True)

In [5]:
file

,time_in,time_out,Duree,NBTRAJTC,dist
0,27631,30107,2476,4,18314.9992865429
1,45297,72546,27249,8,28431.233731882265
2,27665,59922,32257,4,15088.786004118541
3,31487,33789,2302,3,14282.603963737538
4,29194,31996,2802,6,24322.412973969436
5,27767,29732,1965,4,18314.9992865429
6,65773,68806,3033,4,21537.866079156025
7,28506,30632,2126,4,18314.9992865429
8,27982,29903,1921,3,21193.41159429539
9,31319,65274,33955,4,18314.9992865429


In [6]:
data1_true=pd.read_csv('/home/yazi/Documents/stage/data/data1_true.csv',delimiter=';',encoding='iso 8859-1')
data2_true=pd.read_csv('/home/yazi/Documents/stage/data/data2_true.csv',delimiter=';',encoding='iso 8859-1')
data1_false=pd.read_csv('/home/yazi/Documents/stage/data/data1_false.csv',delimiter=';',encoding='iso 8859-1')
data2_false=pd.read_csv('/home/yazi/Documents/stage/data/data2_false.csv',delimiter=';',encoding='iso 8859-1')

In [7]:
Data_Train=pd.concat([data1_true,data2_true,data1_false,data2_false],ignore_index=True)
Data_Train

,Ot,Dt,DUREE,NBTRAJTC,dist,parking
0,18300.0,19800.0,1500.0,1.0,20362.956563,1
1,34200.0,35340.0,1140.0,1.0,1941.648784,1
2,36000.0,37800.0,1800.0,2.0,23648.467181,1
3,27000.0,31800.0,4800.0,3.0,35474.638828,1
4,27000.0,32400.0,5400.0,3.0,35713.582850,1
5,30000.0,32400.0,2400.0,1.0,13400.000000,1
6,27900.0,33600.0,5700.0,3.0,32614.260685,1
7,48600.0,51300.0,2700.0,1.0,16319.620094,1
8,19800.0,22500.0,2700.0,1.0,19009.471324,1
9,30000.0,30300.0,300.0,2.0,100.000000,1


# SVM

In [8]:
X = Data_Train.drop('parking', axis=1)
y = Data_Train['parking']

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-1.5958114 , -1.66829122, -0.57514726, -0.89708632,  0.93431633],
       [-0.77721666, -0.86264201, -0.75408122, -0.89708632, -0.61836558],
       [-0.68454556, -0.73510681, -0.42603562,  0.23129809,  1.21124307],
       ...,
       [ 0.75185653,  0.88241283,  1.21419238,  0.23129809,  2.31540661],
       [ 1.55500608,  1.58230113,  0.17041092,  0.23129809,  2.29709521],
       [ 1.81757421,  1.78449109, -0.42603562, -0.89708632, -0.29098065]])

In [10]:
y=np.array(y.astype(float))
y

array([1., 1., 1., ..., 0., 0., 0.])

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [12]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf',class_weight='balanced')
svclassifier.fit(X_train, y_train)

/home/yazi/anaconda3/envs/stage/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [13]:
y_pred = svclassifier.predict(X_test)

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2914 1182]
 [  36  172]]
              precision    recall  f1-score   support

         0.0       0.99      0.71      0.83      4096
         1.0       0.13      0.83      0.22       208

    accuracy                           0.72      4304
   macro avg       0.56      0.77      0.52      4304
weighted avg       0.95      0.72      0.80      4304



# SVM oversampling

In [15]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=42)
X_res,y_res=sm.fit_resample(X,y)

In [16]:
X_res.shape

(40822, 5)

In [17]:
y_res.shape

(40822,)

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X_res,y_res,test_size=0.2)

In [19]:
svclassifier = SVC(kernel='rbf',class_weight='balanced',probability=True)
svclassifier.fit(X_train, y_train)

/home/yazi/anaconda3/envs/stage/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [20]:
y_pred = svclassifier.predict(X_test)

In [21]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2891 1176]
 [ 687 3411]]
              precision    recall  f1-score   support

         0.0       0.81      0.71      0.76      4067
         1.0       0.74      0.83      0.79      4098

    accuracy                           0.77      8165
   macro avg       0.78      0.77      0.77      8165
weighted avg       0.78      0.77      0.77      8165



In [22]:
svclassifier.predict_proba(X_test)

array([[0.28684342, 0.71315658],
       [0.21557403, 0.78442597],
       [0.88782628, 0.11217372],
       ...,
       [0.87482681, 0.12517319],
       [0.84142649, 0.15857351],
       [0.52260327, 0.47739673]])

In [23]:
svclassifier.predict(X)

array([1., 0., 0., ..., 1., 1., 0.])

# AFC prediction

In [24]:
scaler = StandardScaler()
X0 = scaler.fit_transform(file)
X0

array([[-1.10165604, -1.21512452, -0.17738113,  0.85520398, -0.11471561],
       [-0.12573111,  1.1976657 ,  2.8932032 ,  3.28815296,  0.73442831],
       [-1.09977777,  0.47995172,  3.51393893,  0.85520398, -0.38551987],
       ...,
       [ 0.7737926 ,  0.67654986, -0.26117054,  0.24696674,  2.43363096],
       [ 0.78528316,  0.68837531, -0.26117054,  0.24696674,  2.43363096],
       [ 0.74142013,  0.64323392, -0.26117054,  0.24696674,  2.43363096]])

In [25]:
prediction0=svclassifier.predict(X0)
prediction0

array([0., 0., 0., ..., 1., 1., 1.])

In [26]:
from My_count import *

In [27]:
count(prediction0)

{0.0: 237554, 1.0: 108156}

In [28]:
108156/(108156+237554)

0.3128518122125481

# 30% take car